In [1]:
!rm *.csv
!wget http://smartheatmap.ddns.net/dist/p.csv
!wget http://smartheatmap.ddns.net/dist/n.csv
!pip3 install wldhx.yadisk-direct hyperas hyperopt
!curl -L $(yadisk-direct https://yadi.sk/d/NmzmzI1_v9tecQ) -o w2v.zip
!unzip w2v.zip

rm: cannot remove '*.csv': No such file or directory
--2019-07-17 12:40:01--  http://smartheatmap.ddns.net/dist/p.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27630235 (26M) [text/csv]
Saving to: ‘p.csv’

p.csv               100%[===================>]  26.35M  17.6MB/s    in 1.5s    

2019-07-17 12:40:02 (17.6 MB/s) - ‘p.csv’ saved [27630235/27630235]

--2019-07-17 12:40:04--  http://smartheatmap.ddns.net/dist/n.csv
Resolving smartheatmap.ddns.net (smartheatmap.ddns.net)... 188.120.231.51
Connecting to smartheatmap.ddns.net (smartheatmap.ddns.net)|188.120.231.51|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26037206 (25M) [text/csv]
Saving to: ‘n.csv’

n.csv               100%[===================>]  24.83M  18.2MB/s    in 1.4s    

2019-07-17 12:40:05 (18.2 MB/s) - ‘n.csv’ saved [2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1478M  100 1478M    0     0  44.3M      0  0:00:33  0:00:33 --:--:-- 48.1M
Archive:  w2v.zip
   creating: w2v/
  inflating: w2v/tweets_model.w2v    
  inflating: w2v/tweets_model.w2v.trainables.syn1neg.npy  
  inflating: w2v/tweets_model.w2v.wv.vectors.npy  


In [0]:
import pandas as pd
import numpy as np

data_positive = pd.read_csv('p.csv', sep=';', error_bad_lines=False, usecols=['ttext'])
data_negative = pd.read_csv('n.csv', sep=';', error_bad_lines=False, usecols=['ttext'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['ttext'].values[:sample_size],
                           data_negative['ttext'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [0]:
import re

def clear_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [clear_text(t) for t in raw_data]


In [0]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

SENTENCE_LENGTH = 26
NUM = 100000

tokenizer = Tokenizer(num_words=NUM)
tokenizer.fit_on_texts(x_train)

def to_seq(x):
    sequences = tokenizer.texts_to_sequences(x)
    return pad_sequences(sequences, maxlen=SENTENCE_LENGTH)

with open('tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
  
x_train = to_seq(x_train)
x_test = to_seq(x_test)

Using TensorFlow backend.


In [6]:
from gensim.models import Word2Vec

wvmodel = Word2Vec.load("w2v/tweets_model.w2v")

DIM = wvmodel.vector_size 
embedding_matrix = np.zeros((NUM, DIM))

for word, i in tokenizer.word_index.items():
    if i >= NUM:
        break
    if word in wvmodel.wv.vocab.keys():
        embedding_matrix[i] = wvmodel.wv[word]

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [7]:
from keras.layers import Input
from keras.layers.embeddings import Embedding

m_input = Input(shape=(SENTENCE_LENGTH,), dtype='int32')
# m_embed = Embedding(NUM, 
#                     DIM, 
#                     input_length=SENTENCE_LENGTH,
#                     weights=[embedding_matrix], 
#                     trainable=True)(m_input)
DIM=300
m_embed = Embedding(NUM, 
                    DIM, 
                    input_length=SENTENCE_LENGTH)(m_input)

W0717 12:41:42.540471 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 12:41:42.584094 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 12:41:42.594195 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [8]:
from keras import optimizers
from keras.layers import Dense, concatenate, Activation, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D

x = Dropout(0.2)(m_embed)

conv_branches = []
for size, filters_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
    for i in range(filters_count):
        i = Conv1D(filters=1, 
                   kernel_size=size, 
                   padding='valid', 
                   activation='relu')(x)
        i = GlobalMaxPooling1D()(i)
        conv_branches.append(i)

x = concatenate(conv_branches, 
                axis=1)

x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)

m_output = Activation('sigmoid')(x)

model = Model(inputs=[m_input], 
              outputs=[m_output])

W0717 12:41:42.639526 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0717 12:41:42.662533 139626235393920 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from keras import backend as K


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.
        

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [10]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy',
                       precision, 
                       recall, 
                       f1])

W0717 12:41:43.842788 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0717 12:41:43.874917 139626235393920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0717 12:41:43.885639 139626235393920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
rm cnn*

rm: cannot remove 'cnn*': No such file or directory


In [12]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("cnn-{epoch:02d}-{val_f1:.2f}.hdf5", 
                             monitor='val_f1', 
                             save_best_only=True, 
                             mode='max', 
                             period=1)

history = model.fit(x_train, 
                    y_train, 
                    batch_size=32, 
                    epochs=10, 
                    validation_split=0.25, 
                    callbacks = [checkpoint])

Train on 134307 samples, validate on 44769 samples
Epoch 1/10
134307/134307 [==============================] - 382s 3ms/step - loss: 0.4981 - acc: 0.7463 - precision: 0.7489 - recall: 0.7318 - f1: 0.7324 - val_loss: 0.4560 - val_acc: 0.7765 - val_precision: 0.7862 - val_recall: 0.7596 - val_f1: 0.7669
Epoch 2/10
134307/134307 [==============================] - 368s 3ms/step - loss: 0.3415 - acc: 0.8502 - precision: 0.8555 - recall: 0.8426 - f1: 0.8445 - val_loss: 0.4634 - val_acc: 0.7769 - val_precision: 0.7915 - val_recall: 0.7527 - val_f1: 0.7657
Epoch 3/10
134307/134307 [==============================] - 371s 3ms/step - loss: 0.2087 - acc: 0.9152 - precision: 0.9208 - recall: 0.9084 - f1: 0.9119 - val_loss: 0.5597 - val_acc: 0.7707 - val_precision: 0.7796 - val_recall: 0.7553 - val_f1: 0.7611
Epoch 4/10
134307/134307 [==============================] - 369s 3ms/step - loss: 0.1306 - acc: 0.9490 - precision: 0.9534 - recall: 0.9442 - f1: 0.9471 - val_loss: 0.6858 - val_acc: 0.7621 - v

E0717 13:06:58.427597 139626235393920 ultratb.py:152] Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-3b7c340669eb>", line 14, in <module>
    callbacks = [checkpoint])
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1458, in __call__
    run_metadata_ptr)
KeyboardInterrupt

During handling of

KeyboardInterrupt: ignored

In [0]:
model.predict(to_seq(["Зачем мне подрывать и без того не самую безупречную репутацию кураторов?",
                       "сломанная лавочка",
                       "Ну это же логично, что я дождался, пока ты прочтешь и удалил",
                       "трансгендер на улице",
                       "США",
                       "СССР",
                       "фашист",
                       "на улице поставили хреновую красивую скамейку",
                       "на улице хрено поставили красивую скамейку",
                       "на улице Ленина поставили хреновую скамейку",
                       "красивую скамейку",
                       "мне нравится эта хрень, но будет нереально круто",
                       "Жить конечно тут наверное нет",
                       "Очень-очень красиво, сказочно, солнечно, жарко, волшебно....можно много слов написать, и все они будут про это не забываемое место. Я  не знаю у кого как. Но я очень люблю Сочи-Адлер и люблю тут отдыхать с детьми и одна и с подругами и вообще. Жить конечно тут наверное нет. Но летом сюда-это точно. На все лето, сюда☺☺☺🤩🤩🤩🤩🤩",
                       "эта скамейка очень красивая",
                       "сегодня было грустно",
                       "эта скамейка полная чушь"]))